<a href="https://colab.research.google.com/github/MyDarapy/Named-entity-recognition-in-clinical-texts-using-NLP-techniques/blob/main/extracting_named_medical_entities_from_clinical_texts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U spacy
!pip install scispacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 21.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 8.5 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Created wheel for nmslib: filename=nmslib-2.1.1-cp310-cp310-linux_x86_64.whl size=13578462 sha256=ed1a2819bf0890b92dd06db45e62aeab9d9974d48f608c01ccfcb016130307d7
  Stored in directory: /root/.cache/pip/wheels/21/1a/5d/4cc754a5b1a88405cad184b76f823897a63a8d19afcd4b9314
Successfully built nmslib
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4


In [2]:
!pip install render
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_md-0.5.1.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bc5cdr_md-0.5.1.tar.gz

  Preparing metadata (setup.py) ... done
  Created wheel for render: filename=render-1.0.0-py3-none-any.whl size=3038 sha256=615fa63251737b7d763239b8c41bc088e27352c54bdb9346e16ffd65179afdf8
  Stored in directory: /root/.cache/pip/wheels/05/30/61/64c5e91469799f7e7bfff7c6f9047ab73e856f389f898e9e67
Successfully built render
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.6/919.6 kB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.0 MB/s eta 0:00:00
  Created wheel for en-core-sci-sm: filename=en_core_sci_sm-0.5.1-py3-none-any.whl size=15870855 sha256=50146135308d05874e5716686ea32cd926a7d2936a5912997090b674e71e62c2
  Stored in directory: /root/.cache/pip/wheels/35/8f/33/4c2091669

In [12]:
import scispacy
import spacy
from spacy.matcher import Matcher
import en_core_sci_sm
import en_core_sci_md

#Pretrained named entity recognition specific models
import en_ner_bc5cdr_md

from spacy import displacy
import pandas as pd

In [4]:
mtsample=pd.read_csv("mtsamples.csv")

print(mtsample.shape)

(4999, 6)


In [5]:
transcription = mtsample.loc[55, 'transcription']

In [6]:
nlp_sm = en_core_sci_sm.load()
documment = nlp_sm(transcription)
displacy.render(documment, jupyter= True, style="ent")

nlp_md = en_core_sci_md.load()
documment = nlp_md(transcription)
displacy.render(documment, jupyter = True, style="ent")

In [7]:
#Labels the entity. Model trained to identify and classify entitties that relates to chemicals and diseases in the corpus
nlp_bc = en_ner_bc5cdr_md.load()
documment = nlp_bc(transcription)
displacy.render(documment, jupyter=True, style="ent")

In [8]:
print("TEXT", "START", "END", "ENTITY TYPE")
for ent in documment.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

TEXT START END ENTITY TYPE
hematuria 517 526 DISEASE
right renal stones and patulous right collecting system with notable two right crossing renal arteries. 554 657 DISEASE
povidone-iodine 1402 1417 CHEMICAL
CO2 1612 1615 CHEMICAL


In [9]:
mtsample.dropna(subset=['transcription'], inplace=True)
mtsample_subset_data= mtsample.sample(n=100, replace=False, random_state=42)
mtsample_subset_data.head(2)

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
3162,3162,Markedly elevated PT INR despite stopping Cou...,Hematology - Oncology,Hematology Consult - 1,"HISTORY OF PRESENT ILLNESS:, The patient is w...",NaN
1981,1981,Intercostal block from fourth to tenth interc...,Pain Management,Intercostal block - 1,"PREPROCEDURE DIAGNOSIS:, Chest pain secondary...","pain management, xylocaine, marcaine, intercos..."


In [14]:
''' goal is to locate and to recognize drug names and dosage from a corpus to detect errors
by comparing it with standards and guidelines'''
pattern = [{'ENT_TYPE': 'CHEMICAL'}, {'LIKE_NUM': True}, {'IS_ASCII': True}]
matcher = Matcher(nlp_bc.vocab)
matcher.add('DRUG_DOSAGE', [pattern])
for transcription in mtsample_subset_data['transcription']:
  doc = nlp_bc(transcription)
  matches = matcher(doc)
  for match_id, start, end in matches:
    string_id = nlp_bc.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]                    # The matched span
    print(span.text, start, end, string_id)
    for ent in doc.ents:
      print(ent.text, ent.start_char, ent.end_char, ent.label_)


Xylocaine 20 mL 129 132 DRUG_DOSAGE
Chest pain 26 36 DISEASE
Chest pain 122 132 DISEASE
intercostal block 318 335 DISEASE
chest pain 388 398 DISEASE
Xylocaine 730 739 CHEMICAL
Marcaine 750 758 CHEMICAL
contusion 987 996 DISEASE
respiratory distress 1076 1096 DISEASE
pain 1150 1154 DISEASE
Marcaine 0.25% 133 136 DRUG_DOSAGE
Chest pain 26 36 DISEASE
Chest pain 122 132 DISEASE
intercostal block 318 335 DISEASE
chest pain 388 398 DISEASE
Xylocaine 730 739 CHEMICAL
Marcaine 750 758 CHEMICAL
contusion 987 996 DISEASE
respiratory distress 1076 1096 DISEASE
pain 1150 1154 DISEASE
Aspirin 81 mg 204 207 DRUG_DOSAGE
non-small cell lung cancer 114 140 DISEASE
barium 322 328 CHEMICAL
hiatal hernia 373 386 DISEASE
odynophagia 647 658 DISEASE
tenderness 829 839 DISEASE
DVT 918 921 DISEASE
weight loss 952 963 DISEASE
anorexia 965 973 DISEASE
fevers 975 981 DISEASE
chills 983 989 DISEASE
headaches 991 1000 DISEASE
aches 1006 1011 DISEASE
pains 1015 1020 DISEASE
cough 1022 1027 DISEASE
hemoptysis 1029 1